In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.tokenize import RegexpTokenizer

In [ ]:
df_all = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx')

# EDA

In [ ]:
df_all = df_all.drop(columns = ['Unnamed: 0'])

In [ ]:
df_all.info()

In [ ]:
pd.DataFrame(df_all[['title', 'abstract']])

In [ ]:
topik_riset = {'phew'}

for i in range(len(df_all['fokus_riset'])):
  topik_riset.add(df_all['fokus_riset'].iloc[i])

In [ ]:
topik_riset

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if df_all['fokus_riset'].iloc[i] == 'Kesehatan-Obat':
    temporal.append(df_all.iloc[i])

In [ ]:
temporal = []

for i in range(len(df_all['fokus_riset'])):
  if pd.isnull(df_all['fokus_riset'].iloc[i]):
    temporal.append(df_all.iloc[i])

In [ ]:
len(temporal)

In [ ]:
df_temp = pd.DataFrame(temporal)
df_temp[['title', 'fokus_riset']]

In [ ]:
df_all['fokus_riset'].iloc[3921] = 'Kesehatan-Obat'
df_all['fokus_riset'].iloc[3926] = 'Sosial Humaniora, Seni Budaya, Pendidikan'

# Mesin Pencari

In [ ]:
df_all.info()

In [ ]:
df_all['title'].iloc[5]

In [ ]:
# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

In [ ]:
# Contoh teks
text1 = 'Tingkat METIL METSULFURON dalam COpedoda dhipa sp hahahaha'
text2 = 'Tingkat METIL METSULFURON dalam TOKSISITAS MASKULINITAS COPEPODA Daphnia sp.'

# Inisialisasi TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Transformasi teks menjadi vektor TF-IDF
tfidf_matrix = tfidf_vectorizer.fit_transform([text1, text2])

# Hitung similaritas kosinus antara kedua teks
cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

# Tampilkan hasil similaritas
print("Similaritas antara teks 1 dan teks 2:", cosine_similarities[0][0])

In [ ]:
dicari = 'hutan kawasan lindung'
dicari = dicari.lower()

In [ ]:
def remove_stopwords(text):
    factory = StopWordRemoverFactory()
    stopwords = factory.get_stop_words()

    result = []
    for x in  text.split():
        if x not in stopwords:
            result.append(x)

    hasil_stopword = " ".join(result)

    return hasil_stopword

In [ ]:
def stem(text):
    factory2 = StemmerFactory()
    stemmer = factory2.create_stemmer()
    return stemmer.stem(text)

In [ ]:
dicari = stem(dicari)
dicari

In [ ]:
from tqdm import tqdm

df_temu = []
similaritas = []

for i in tqdm(range(len(df_all['text_stemmed']))):
  temp = df_all['text_stemmed'].iloc[i]

  # Transformasi teks menjadi vektor TF-IDF
  tfidf_matrix = tfidf_vectorizer.fit_transform([dicari, temp])

  # Hitung similaritas kosinus antara kedua teks
  cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

  if cosine_similarities[0][0] > 0.1:
    df_temu.append(df_all.iloc[i])
    similaritas.append(cosine_similarities)

df_temu = pd.DataFrame(df_temu)
df_temu['similaritas'] = similaritas

In [ ]:
len(df_temu)

In [ ]:
df_temu[['title', 'similaritas','abstract']].sort_values(by=['similaritas'], ascending=False).head(10)

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

topik = []

for index in range(len(df_temu)):
  teks = df_temu['fulltext_stopword'].iloc[index]
  data = tokenizer.tokenize(teks)

  # Proses vektorisasi teks dengan TF-IDF
  tfidf_vectorizer = TfidfVectorizer()
  tfidf_matrix = tfidf_vectorizer.fit_transform(data)

  # Terapkan LSA dengan TruncatedSVD
  if len(data) <= 5:
     num_topics = len(data) # Jumlah topik yang ingin diidentifikasi
  else:
    num_topics = 5  # Jumlah topik yang ingin diidentifikasi
  lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
  lsa_model.fit_transform(tfidf_matrix)

  # Ambil kata-kata kunci untuk setiap topik
  terms = tfidf_vectorizer.get_feature_names_out()
  top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

  kata_kunci = []

  for i, topic in enumerate(lsa_model.components_):
      top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
      keywords = [terms[key_idx] for key_idx in top_keywords_idx]
      kata_kunci.append(keywords[0])

  topik.append(kata_kunci)

df_temu['topik'] = topik

In [ ]:
df_temu[['title', 'similaritas','abstract','keywords','topik']].sort_values(by=['similaritas'], ascending=False)

# Test SCRIPT STKI

In [22]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.tokenize import RegexpTokenizer
from tqdm import tqdm

def search(teks, pilihan: int = 0):
  df = pd.read_excel(r'D:\TEST Codingan\Sistem-Temu-Kembali-Informasi\Riset Jupiter Notebook\Data\df_final.xlsx')
  df = df.drop(columns='Unnamed: 0')

  teks = stem(teks)

  df_temu = []
  similaritas = []

  # Inisialisasi TF-IDF Vectorizer
  tfidf_vectorizer = TfidfVectorizer()

  for i in tqdm(range(len(df['text_stemmed']))):
    temp = df['text_stemmed'].iloc[i]

    # Transformasi teks menjadi vektor TF-IDF
    tfidf_matrix = tfidf_vectorizer.fit_transform([teks, temp])

    # Hitung similaritas kosinus antara kedua teks
    cosine_similarities = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])

    if cosine_similarities[0][0] > 0.1:
      df_temu.append(df.iloc[i])
      similaritas.append(cosine_similarities[0, 0])

  df_temu = pd.DataFrame(df_temu)
  df_temu['similaritas'] = similaritas
  return tentukan_topik(df_temu.sort_values(by=['similaritas'], ascending=False), pilihan)

def remove_stopwords(text):
  factory = StopWordRemoverFactory()
  stopwords = factory.get_stop_words()

  result = []
  for x in  text.split():
      if x not in stopwords:
          result.append(x)

  hasil_stopword = " ".join(result)

  return hasil_stopword

def stem(text):
  factory2 = StemmerFactory()
  stemmer = factory2.create_stemmer()
  return stemmer.stem(text)

def hapus_nan(dataset):
  dataset.fillna("", inplace=True) 
  return dataset

def tentukan_topik(dataset, pilihan:int = 0):
  tokenizer = RegexpTokenizer(r'\w+')
  topik = []

  for index in range(len(dataset)):
    # Ambil dataset
    teks = dataset['fulltext_stopword'].iloc[index]
    data = tokenizer.tokenize(teks)

    # Proses vektorisasi teks dengan TF-IDF
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(data)
    jumlah_feature = tfidf_matrix.shape

    # Terapkan LSA dengan TruncatedSVD
    if jumlah_feature[1] < 5:
      num_topics = jumlah_feature[1] # Jumlah topik yang ingin diidentifikasi
    else:
      num_topics = 5 # Jumlah topik yang ingin diidentifikasi
    lsa_model = TruncatedSVD(n_components=num_topics, random_state=42)
    lsa_model.fit_transform(tfidf_matrix)

    # Ambil kata-kata kunci untuk setiap topik
    terms = tfidf_vectorizer.get_feature_names_out()
    top_keywords = 1  # Jumlah kata kunci untuk setiap topik (HANYA BISA DIAMBIL 1)

    kata_kunci = []

    for i, topic in enumerate(lsa_model.components_):
        top_keywords_idx = topic.argsort()[-top_keywords:][::-1]
        keywords = [terms[key_idx] for key_idx in top_keywords_idx]
        kata_kunci.append(keywords[0])

    topik.append(kata_kunci)
  dataset['topik'] = topik
  dataset = dataset.drop(columns=[
     'text_stemmed',
     'fulltext_stopword', 
     'fulltext', 
     'language_accuracy', 
     'language', 
     'text_raw',
     'abstract', 
    #  'date',
    #  'keywords',
    #  'similaritas',
    #  'new_abstract',
    #  'publisher',
    #  'publication',
    #  'divisions'
    ])
  
  dataset = hapus_nan(dataset) #hapus nilai nan
  if pilihan == 0:
    return convert_dataframe_to_list(dataset)
  elif pilihan == 1:
    return dataset
  elif pilihan == 2:
    return convert_dataframe_to_dict(dataset)

def convert_dataframe_to_list(dataset):
  dataset = dataset.T
  hasil_pencarian = []
  for i in dataset.columns.values.tolist():
    hasil_pencarian.append(dataset[i].to_list())
  return hasil_pencarian

def convert_dataframe_to_dict(dataset):
  dataset = dataset.T
  hasil_pencarian = dataset.to_dict()
  return hasil_pencarian

In [23]:
df = search('Penerapan dan Pendampingan Peningkatkan Kesehatan Sanitasi', 1)

100%|██████████| 15867/15867 [01:03<00:00, 250.45it/s]


In [24]:
df[['uri','subjects','title','similaritas']]

,uri,subjects,title,similaritas
3321,http://repository.lppm.unila.ac.id/id/eprint/2...,['QA'],Penerapan dan Pendampingan Peningkatkan Keseha...,0.293768
3320,http://repository.lppm.unila.ac.id/id/eprint/2...,['QA'],Penerapan dan Pendampingan Peningkatkan Keseha...,0.293768
15522,http://repository.lppm.unila.ac.id/id/eprint/4...,['L1'],Peningkatan Perilaku Hidup Bersih dan Sehat (P...,0.257637
15521,http://repository.lppm.unila.ac.id/id/eprint/4...,['Q1'],PENINGKATAN PERILAKU HIDUP BERSIH DAN SEHAT (P...,0.250818
3558,http://repository.lppm.unila.ac.id/id/eprint/2...,['R1'],"Penerapan eKIE (Komunikasi, Informasi, dan Edu...",0.227289
...,...,...,...,...
4832,http://repository.lppm.unila.ac.id/id/eprint/2...,['R1'],Pemberdayaan Siswa sebagai Peer Educator dalam...,0.101384
13158,http://repository.lppm.unila.ac.id/id/eprint/4...,['SF'],"PENYULUHAN MANAJEMEN KESEHATAN, PERKANDANGAN, ...",0.101292
3703,http://repository.lppm.unila.ac.id/id/eprint/2...,['SD'],PENTINGNYA PEMANTAUAN KESEHATAN HUTAN DALAM PE...,0.100870
4410,http://repository.lppm.unila.ac.id/id/eprint/2...,['SD'],Kajian Kesehatan Hutan Dalam Pengelolaan Hutan...,0.100720


In [25]:
df['uri']

3321     http://repository.lppm.unila.ac.id/id/eprint/2...
3320     http://repository.lppm.unila.ac.id/id/eprint/2...
15522    http://repository.lppm.unila.ac.id/id/eprint/4...
15521    http://repository.lppm.unila.ac.id/id/eprint/4...
3558     http://repository.lppm.unila.ac.id/id/eprint/2...
                               ...                        
4832     http://repository.lppm.unila.ac.id/id/eprint/2...
13158    http://repository.lppm.unila.ac.id/id/eprint/4...
3703     http://repository.lppm.unila.ac.id/id/eprint/2...
4410     http://repository.lppm.unila.ac.id/id/eprint/2...
4409     http://repository.lppm.unila.ac.id/id/eprint/2...
Name: uri, Length: 191, dtype: object

In [18]:
df['subjects'].iloc[1]

"['G1', 'GA', 'GB', 'GE', 'Q1']"